<a href="https://colab.research.google.com/github/thisissamuca/GOES_16/blob/main/CUT_MS_MG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usado para obter a extenção do Mato Grosso do Sul e do Mato Grosso

In [ ]:
pip install rasterio fiona geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 28.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Arquivo único

In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas
import numpy as np

def clip_raster_with_shapefile(raster_path, shapefile_path, output_path):
    """
    Recorta um raster utilizando um shapefile e reclassifica valores 0 para nan.

    Args:
        raster_path (str): Caminho para o arquivo raster.
        shapefile_path (str): Caminho para o arquivo shapefile.
        output_path (str): Caminho para salvar o raster recortado.
    """
    try:
        # Abrir o raster
        with rasterio.open(raster_path) as src:
            # Ler o shapefile
            gdf = geopandas.read_file(shapefile_path)

            # Garantir que o shapefile esteja no mesmo CRS que o raster
            if gdf.crs != src.crs:
                gdf = gdf.to_crs(src.crs)

            # Converter o GeoDataFrame para lista de geometrias para a função mask
            shapes = [geometry for geometry in gdf.geometry]

            # Recortar o raster
            out_image, out_transform = mask(src, shapes=shapes, crop=True)
            out_meta = src.meta.copy()

            # Reclassificar valores 0 para nan
            out_image = out_image.astype(np.float32)  # Converter para float para permitir nan
            out_image[out_image == 0] = np.nan

            # Atualizar metadados para o raster recortado
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
                "dtype": 'float32',  # Atualizar o tipo de dados
                "nodata": np.nan     # Definir nan como valor nodata
            })

            # Salvar o raster recortado
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)

        print(f"Raster recortado com sucesso e salvo em: {output_path}")

    except Exception as e:
        print(f"Ocorreu um erro ao recortar o raster: {e}")

# Exemplo de uso:
raster_path = "/content/ABI-L2-CMIPF-M6C01_G16_s20201521300164_e20201521309472_c20201521309546.tif"
shapefile_path = "/content/MS_MG.shp"
output_path = "/content/arquivo.tif"

clip_raster_with_shapefile(raster_path, shapefile_path, output_path)

Raster recortado com sucesso e salvo em: /content/arquivo.tif


# Lote de arquivos

In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas
import numpy as np
import os
from tqdm import tqdm  # para barra de progresso (opcional)

def process_raster_batch(input_dir, shapefile_path, output_dir):
    """
    Processa em massa arquivos .tif de um diretório, recortando com um shapefile e reclassificando 0 para nan.

    Args:
        input_dir (str): Caminho para o diretório com os arquivos .tif de entrada
        shapefile_path (str): Caminho para o arquivo shapefile (o mesmo para todos)
        output_dir (str): Caminho para o diretório de saída
    """
    try:
        # Verificar/Criar diretório de saída
        os.makedirs(output_dir, exist_ok=True)

        # Carregar o shapefile uma única vez (para todos os rasters)
        gdf = geopandas.read_file(shapefile_path)

        # Listar todos os arquivos .tif no diretório de entrada
        tif_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.tif')]

        if not tif_files:
            print("Nenhum arquivo .tif encontrado no diretório de entrada.")
            return

        print(f"Iniciando processamento de {len(tif_files)} arquivos...")

        for tif_file in tqdm(tif_files, desc="Processando rasters"):
            try:
                raster_path = os.path.join(input_dir, tif_file)
                output_path = os.path.join(output_dir, f"clip_{tif_file}")

                # Abrir o raster
                with rasterio.open(raster_path) as src:
                    # Garantir que o shapefile esteja no mesmo CRS que o raster atual
                    if gdf.crs != src.crs:
                        current_gdf = gdf.to_crs(src.crs)
                    else:
                        current_gdf = gdf

                    # Converter para lista de geometrias
                    shapes = [geometry for geometry in current_gdf.geometry]

                    # Recortar o raster
                    out_image, out_transform = mask(src, shapes=shapes, crop=True)
                    out_meta = src.meta.copy()

                    # Reclassificar valores 0 para nan
                    out_image = out_image.astype(np.float32)
                    out_image[out_image == 0] = np.nan

                    # Atualizar metadados
                    out_meta.update({
                        "driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform,
                        "dtype": 'float32',
                        "nodata": np.nan
                    })

                    # Salvar o raster recortado
                    with rasterio.open(output_path, "w", **out_meta) as dest:
                        dest.write(out_image)

            except Exception as e:
                print(f"\nErro ao processar {tif_file}: {e}")
                continue

        print(f"\nProcessamento concluído! Arquivos salvos em: {output_dir}")

    except Exception as e:
        print(f"Ocorreu um erro geral: {e}")

# Exemplo de uso:
input_directory = "/content/drive/MyDrive/NBR/ARQUIVOS_PROCESSADOS/MASCARA_NUVEM_APLICADA"
shapefile_path = "/content/drive/MyDrive/NBR/ARQUIVOS_BRUTOS/SHAPEFILES/RECORTE_MS_MG/MS_MG.shp"
output_directory = "/content/drive/MyDrive/NBR/ARQUIVOS_PROCESSADOS/RECORTE_DADOS"

process_raster_batch(input_directory, shapefile_path, output_directory)

Iniciando processamento de 60 arquivos...


Processando rasters: 100%|██████████| 60/60 [00:59<00:00,  1.01it/s]


Processamento concluído! Arquivos salvos em: /content/drive/MyDrive/NBR/ARQUIVOS_PROCESSADOS/RECORTE_DADOS
